In [ ]:
## Transferring files with checkpointing

In [ ]:
## STEP 1: Create list of unique folders on the destination machine corresponding to the files being transferred

# get list of unique folders on source machine
!find . -name "*" |  awk -F'/' '{for (i=1; i<NF; i++) printf("%s/", $i);print "\n"}' | sort | uniq > unique_dirs.txt

# create identical directory structure on the destination machine
!cat unique_dirs.txt | xargs mkdir -p

In [ ]:
## STEP 2: Get list of all files to be copied from the source machine, also create an initial empty checkpoint file

!find . -name "*" |  awk -F'/' '{for (i=1; i<NF; i++) printf("%s/", $i);print "\n"} > filelist.txt
!touch checkpoint.txt 

In [36]:
%%writefile transfer_files.py

## STEP 3: Copy files over with checkpointing support. 
## Checkpointing remembers the last file transferred so that a failed transfer can be resumed

import sys
import subprocess

privateKey = sys.argv[1]
serverInfo = sys.argv[2]
sourceDir = sys.argv[3]

print("Transferring files using %s and %s. Source dir:%s" %  (privateKey, serverInfo,sourceDir))

fList = open("fileslist.txt")
fCheckpoint = open("checkpoint.txt")
lastFile = fCheckpoint.read().strip()
fCheckpoint.close()

checkPointFound = False
for file in fList:
        
    file = file.strip()
    
    #print "Last File:%s,len(lastFile):%d" % (lastFile,len(lastFile))
    #print "File:%s,len(file):%d" % (file,len(file))

    # check to see if we are at directory
    fileName = file.split("/")[-1]
    if "." not in fileName:
        print "Skipping directory:%s" % file
        continue
    
    if checkPointFound == False:
        if lastFile == "":
            print "Empty checkpoint file."
            checkPointFound = True
        elif lastFile != file:
            print "Skipping file. Checkpoint not reached"
            continue
        else:
            print "Checkpoint reached. Starting to process files"
            checkPointFound = True
    
    # add file to checkpoint
    command = "scp -i " + privateKey + " " + sourceDir + file + " " + serverInfo + file
    
    # print command
    subprocess.call(command,shell=True)
    
    # add to checkpoint
    fCheckpoint = open("checkpoint.txt","w")
    fCheckpoint.write(file)
    fCheckpoint.close()


    

Overwriting transfer_files.py
